## Creating a MCP Client

### Building your own MCP
#### This will run inside our HOST or Our own Application

#### Reference Code
``` python
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="uv",  # Executable
    args=["run example_server.py"],  # Command line arguments
    env=None,  # Optional environment variables
)

async def run():
    # Launch the server as a subprocess & returns the read and write streams
    # read: the stream that the client will use to read msgs from the server
    # write: the stream that client will use to write msgs to the server
    async with stdio_client(server_params) as (read, write): 
        # the client session is used to initiate the connection 
        # and send requests to server 
        async with ClientSession(read, write) as session:
            # Initialize the connection (1:1 connection with the server)
            await session.initialize()

            # List available tools
            tools = await session.list_tools()

            # will call the chat_loop here
            # ....
            
            # Call a tool: this will be in the process_query method
            result = await session.call_tool("tool-name", arguments={"arg1": "value"})


if __name__ == "__main__":
    asyncio.run(run())
`````

Here're the `mcp_chatbot` code `Anthropic`.

In [10]:
%%writefile mcp_project/anthropic_mcp_chatbot.py
from dotenv import load_dotenv
from anthropic import Anthropic
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List
import asyncio
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.session: ClientSession = None
        self.anthropic = Anthropic()
        self.available_tools: List[dict] = []

    async def process_query(self, query):
        messages = [{'role':'user', 'content':query}]
        response = self.anthropic.messages.create(mapx_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219', 
                                      tools = self.available_tools, # tools exposed to the LLM
                                      messages = messages)
        process_query = True
        while process_query:
            assistant_content = []
            for content in response.content:
                if content.type =='text':
                    print(content.text)
                    assistant_content.append(content)
                    if(len(response.content) == 1):
                        process_query= False
                elif content.type == 'tool_use':
                    assistant_content.append(content)
                    messages.append({'role':'assistant', 'content':assistant_content})
                    tool_id = content.id
                    tool_args = content.input
                    tool_name = content.name
    
                    print(f"Calling tool {tool_name} with args {tool_args}")
                    
                    # Call a tool
                    #result = execute_tool(tool_name, tool_args): not anymore needed
                    # tool invocation through the client session
                    result = await self.session.call_tool(tool_name, arguments=tool_args)
                    messages.append({"role": "user", 
                                      "content": [
                                          {
                                              "type": "tool_result",
                                              "tool_use_id":tool_id,
                                              "content": result.content
                                          }
                                      ]
                                    })
                    response = self.anthropic.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219', 
                                      tools = self.available_tools,
                                      messages = messages) 
                    
                    if(len(response.content) == 1 and response.content[0].type == "text"):
                        print(response.content[0].text)
                        process_query= False

    
    
    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        
        while True:
            try:
                query = input("\nQuery: ").strip()
        
                if query.lower() == 'quit':
                    break
                    
                await self.process_query(query)
                print("\n")
                    
            except Exception as e:
                print(f"\nError: {str(e)}")
    
    async def connect_to_server_and_run(self):
        # Create server parameters for stdio connection
        server_params = StdioServerParameters(
            command="uv",  # Executable
            args=["run", "research_server.py"],  # Optional command line arguments
            env=None,  # Optional environment variables
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                # Initialize the connection
                await session.initialize()
    
                # List available tools
                response = await session.list_tools()
                
                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])
                
                self.available_tools = [{
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                } for tool in response.tools]
    
                await self.chat_loop()


async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()
  

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/anthropic_mcp_chatbot.py


Here're the `mcp_chatbot` code `OpenAI`.

In [ ]:
%%writefile mcp_project/openai_mcp_chatbot.py
import json
from dotenv import load_dotenv
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import List
import asyncio
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.session: ClientSession = None
        self.openai = OpenAI()
        self.available_tools: List[dict] = []


    async def process_query(self, query: str):
        messages = [{"role": "user", "content": query}]

        is_loop_nedded = True
        while is_loop_nedded:
            response = self.openai.responses.create(
                model="gpt-4o",
                input=messages,
                tools=self.available_tools,
            )
            for block in response.output:

                if block.type == "message":
                    is_loop_nedded = False
                    return block.content[0].text

                elif block.type == "function_call":
                    tool_call_id = block.call_id
                    tool_name = block.name
                    tool_args = json.loads(block.arguments)

                    print(
                        f"Processing tool call: {tool_name} with args: {tool_args} with call_id: {tool_call_id}"
                    )
                    # Call a tool
                    # result = execute_tool(tool_name, tool_args)
                    # tool invocation through the client session
                    result = await self.session.call_tool(tool_name, arguments=tool_args)
                    messages.append(block.model_dump())
                    messages.append(
                        {
                            "type": "function_call_output",
                            "call_id": tool_call_id,
                            "output": str(result),
                        }
                    )

                    is_loop_nedded = True

    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")

        while True:
            try:
                query = input("\nQuery :[Type 'quit' to exit] ")
                if not query.strip():
                    print("Query cannot be empty. Please try again.")
                    continue

                if query.lower() == 'quit':
                    break

                final_response = await self.process_query(query)
                print(f"Response : {final_response}\n")

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def connect_to_server_and_run(self):
        # Create server parameters for stdio connection
        server_params = StdioServerParameters(
            command="uv",  # Executable
            args=["run", "research_server.py"],  # Optional command line arguments
            env=None,  # Optional environment variables
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                # Initialize the connection
                await session.initialize()

                # List available tools
                response = await session.list_tools()

                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])

                self.available_tools = [{
                    "type": "function",
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema,
                } for tool in response.tools]

                # Add the "additionalProperties": False : For OpenAI compatibility
                for tool in self.available_tools:
                    tool["parameters"]["additionalProperties"] = False

                await self.chat_loop()


async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()


if __name__ == "__main__":
    asyncio.run(main())

Writing mcp_project/openai_mcp_chatbot.py


## Run the MCP Client and Server

```bash
# Already we have the Virtual Environment created
cd mcp_project
source .venv/bin/activate

# Install the required packages
uv add anthropic openai python-dotenv nest_asyncio

# Terminal : Run the MCP Client
uv run openai_mcp_chatbot.py # This is our End Working ChatBot Which is Interacting with the MCP Server for Tools

# Finally you can Able to Ask the Question and get the tools response with the AI Model
```

### This is the response from our chatbot using the MCP server tools to run and interact with AI models

Here are some papers related to AI in Psychology for understanding and normalizing human emotions:

1. **The Real Her? Exploring Whether Young Adults Accept Human-AI Love**
   - **Authors**: Shuning Zhang, Shixuan Li
   - **Summary**: This paper explores the acceptance of human-AI relationships among young adults, focusing on emotional interactions with AI companions and their implications.
   - [Read more](http://arxiv.org/pdf/2503.03067v1)
   - **Published**: 2025-03-05

2. **The Good, The Bad, and Why: Unveiling Emotions in Generative AI**
   - **Authors**: Cheng Li, Jindong Wang, Yixuan Zhang, Kaijie Zhu, Xinyi Wang, Wenxin Hou, Jianxun Lian, Fang Luo, Qiang Yang, Xing Xie
   - **Summary**: This paper delves into how generative AI models comprehend emotions and employs approaches like EmotionPrompt and EmotionDecode for better AI performance.
   - [Read more](http://arxiv.org/pdf/2312.11111v3)
   - **Published**: 2023-12-18

3. **AI shares emotion with humans across languages and cultures**
   - **Authors**: Xiuwen Wu, Hao Wang, Zhiang Yan, Xiaohan Tang, Pengfei Xu, Wai-Ting Siok, Ping Li, Jia-Hong Gao, Bingjiang Lyu, Lang Qin
   - **Summary**: This study investigates how AI systems align with human emotional perception across different languages and cultures, demonstrating AI's structural emotional congruence with humans.
   - [Read more](http://arxiv.org/pdf/2506.13978v1)
   - **Published**: 2025-06-11

4. **Human-like Affective Cognition in Foundation Models**
   - **Authors**: Kanishk Gandhi, Zoe Lynch, Jan-Philipp Fränken, Kayla Patterson, Sharon Wambu, Tobias Gerstenberg, Desmond C. Ong, Noah D. Goodman
   - **Summary**: This paper evaluates foundation models' abilities to mimic human-like emotional cognition, finding that they can often predict human emotional judgments accurately.
   - [Read more](http://arxiv.org/pdf/2409.11733v2)
   - **Published**: 2024-09-18

5. **Emotion Recognition in Conversation: Research Challenges, Datasets, and Recent Advances**
   - **Authors**: Soujanya Poria, Navonil Majumder, Rada Mihalcea, Eduard Hovy
   - **Summary**: This paper discusses advances and challenges in emotion recognition in conversations, emphasizing the need for scalable algorithms for emotion-aware dialogues.
   - [Read more](http://arxiv.org/pdf/1905.02947v1)
   - **Published**: 2019-05-08

These papers offer insights into how AI might understand and interact with human emotions.